In [4]:
import numpy as np 
import xarray as xr
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from imblearn.over_sampling import SMOTE


In [5]:
tr = range(2007,2008)
vl = range(2008,2009)
te = range(2009,2010)
month = 5
# feature data

dir1 = 'N:/hpc/Data/'
dir2 = 'H:/DeepLear/Data/'
dir3 = 'H:/Explortory/Data/'


In [6]:
def predict1(dirk,data,var1,var2,generate_tensor=  False):
    file_path = dirk + data +'.nc'
    if generate_tensor == True:
        ds2 = xr.open_dataset(file_path ,drop_variables="geopotential", engine='netcdf4')
    else:
        ds2 = xr.open_dataset(file_path , engine='netcdf4')
    ds2  = ds2.sel(time=ds2.time.dt.month.isin([month]))   
    # get actual timesteps   
    actual_days = ds2.time.values 
    
    # get Month-Day of each timestep
    dates_grouped = pd.to_datetime(ds2.time.values).strftime('%m%d')      
    
    # 5-day smoothed climatology. Rolling can be applied directly because the daily data refer to consequtive days. If
    # days are not consecutive, firstly the xr.resample should be applied, so that missing days are generated with NaN
    Smoothed = ds2.rolling(time=5, center=True, min_periods=1).mean() # 5-day smoothing
    
    # change the time to Month-Day
    ds2 = ds2.assign_coords({'time': dates_grouped}) 
    
    # change the time to Month-Day
    Smoothed = Smoothed.assign_coords({'time': dates_grouped}) 
      
    # climatology of the smoothed data 
    Climatology = Smoothed.groupby('time').mean() 
    #If we do not want 5 day moving window
    #Climatology = Daily.groupby('time').mean() 
   
    #sutract the climatology   
    Anomalies = ds2.groupby('time') - Climatology
    
    # change back to the original timestep information
    Anomalies = Anomalies.assign_coords({'time': actual_days}) 
    Anomalies = Anomalies.rename({var1: var2})
    return Anomalies


In [7]:
z500 = predict1(dir3, "Z500", 'daily','Z500', generate_tensor = True)
#z1000 = predict1(dir1, "era5_z1000_day_na_1979-2020", 'geopotential','Z1000')
#z300 = predict1(dir1, "era5_z300_day_na_1979-2020", 'geopotential','Z300')
#t500 = predict1(dir2, "era5_t500_day_na_1979-2020", 'ta','t500')
#t850 = predict1(dir2, "era5_t850_day_na_1979-2020", 'ta','t850')


In [8]:
z500

<xarray.Dataset>
Dimensions:  (time: 1302, lon: 481, lat: 281)
Coordinates:
  * time     (time) datetime64[ns] 1979-05-01 1979-05-02 ... 2020-05-31
  * lon      (lon) float64 -80.0 -79.75 -79.5 -79.25 ... 39.25 39.5 39.75 40.0
  * lat      (lat) float64 20.0 20.25 20.5 20.75 21.0 ... 89.25 89.5 89.75 90.0
Data variables:
    Z500     (time, lat, lon) float64 -19.48 -19.09 -18.73 ... 145.2 145.2 145.2

In [9]:
ds7 = xr.merge([z500])
x_train  = ds7.sel(time=ds7.time.dt.year.isin([tr]))
x_val  = ds7.sel(time=ds7.time.dt.year.isin([vl]))
x_test  = ds7.sel(time=ds7.time.dt.year.isin([te]))


In [10]:
x_train.to_netcdf("N:/hpc/Data/x_train.nc") # save data used for plotting
x_val.to_netcdf("N:/hpc/Data/x_val.nc") # save data used for plotting
x_test.to_netcdf("N:/hpc/Data/x_test.nc") # save data used for plotting

In [11]:
x_val

<xarray.Dataset>
Dimensions:  (time: 31, lon: 481, lat: 281)
Coordinates:
  * time     (time) datetime64[ns] 2008-05-01 2008-05-02 ... 2008-05-31
  * lon      (lon) float64 -80.0 -79.75 -79.5 -79.25 ... 39.25 39.5 39.75 40.0
  * lat      (lat) float64 20.0 20.25 20.5 20.75 21.0 ... 89.25 89.5 89.75 90.0
Data variables:
    Z500     (time, lat, lon) float64 -13.07 -13.15 -13.26 ... -48.53 -48.53

In [12]:
df = pd.read_csv('H:\Explortory\Data\PCS.csv')
df1 = df.assign(year = df["date"].str[:4])
df1 = df1.assign(month = df["date"].str[6:7])
df2 =  df1[df1["month"] == "5"]
df2 = df2[["extreme","year"]]
df2.loc[df2['extreme']== "Yes", 'ex'] = 1 
df2.loc[df2['extreme']== "No", 'ex'] = 0


In [ ]:
df2.head()

In [13]:
years_to_train = ["2007"]
years_to_valid = ["2008"]
years_to_test = ["2009"]


y_train = df2[df2.year.isin(years_to_train)]
y_val = df2[df2.year.isin(years_to_valid)]

y_test = df2[df2.year.isin(years_to_test)]

In [14]:
y_train.to_csv("N:/hpc/Data/y_train.csv") 
y_val.to_csv("N:/hpc/Data/y_val.csv") 
y_test.to_csv("N:/hpc/Data/y_test.csv") 

In [ ]:
dir_loc = 'N:/hpc/Data/'


In [ ]:
y_train

In [ ]:

import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,Conv2D,Add
from tensorflow.keras.layers import SeparableConv2D,ReLU
from tensorflow.keras.layers import BatchNormalization,MaxPool2D
from tensorflow.keras.layers import GlobalAvgPool2D
from tensorflow.keras import Model
import numpy
import netCDF4 as nc
import xarray
import tensorflow_datasets as tfds
import os
import shutil
import glob
#import plotly.express as px
from tensorflow import keras
import pandas as pd
import csv
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import AUC
import matplotlib.pyplot as plt

In [ ]:
def simple_load_nc_dir_with_generator(data, generate_tensor=True):
    #def gen():
        results_list = list()
        final_dict = None
        file_path = dir_loc + data +'.nc'
        
        ds = xarray.open_dataset(file_path, engine='netcdf4')

        if generate_tensor == True:
           
            results_list += list(map( lambda x : tf.convert_to_tensor(x[1]) ,
                                     ds.items() ))
        else:
            
            results_list += list(map( lambda x : x[1],
                                     ds.items() ))

        print("CCC", ds.items())

        
        results_list_shape = results_list[0].shape   
        final_dict = { 
                    'conv2d_input': tf.reshape( results_list[0:1],
                                               shape=(results_list_shape[0],
                                                      results_list_shape[1], 
                                                      results_list_shape[2], 1)) }

      

        #print("BBB", results_list[0] )
        print("X # of inputs:", final_dict['conv2d_input'].shape, "and", type(final_dict['conv2d_input']))

        return final_dict


In [ ]:
X_values = simple_load_nc_dir_with_generator("x_train", generate_tensor=True)
print(type(X_values))
X_val = simple_load_nc_dir_with_generator("x_val", generate_tensor=True)
X_val['conv2d_input'].shape
X_test = simple_load_nc_dir_with_generator("x_test", generate_tensor=True)


In [ ]:
X_val['conv2d_input'].shape

In [ ]:
def open_csv_file(data):
   
    Y_values = list()
    with open(dir_loc + data) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:
                #print(f'\t{row[0]} and {row[1]} and {row[2]}.')
                if row[1] == "Yes":
                    Y_values.append(1)
                else:
                    Y_values.append(0)

                line_count += 1
        print(f'Processed {line_count} lines.')

   
    Y_values = list(map(lambda x : tf.convert_to_tensor(x), Y_values))

    Y_values = {'dense_3': tf.convert_to_tensor( Y_values ) }


    return Y_values

In [ ]:
Y_values = open_csv_file("y_train.csv") 
Y_val = open_csv_file("y_val.csv") 
Y_values['dense_3'].shape
Y_test = open_csv_file("y_test.csv") 

In [ ]:
Y_val['dense_3'].shape


In [ ]:
from tensorflow.keras.layers import Input,Dense,Conv2D,Add,Flatten,Dropout
from tensorflow.keras.layers import SeparableConv2D,ReLU
from tensorflow.keras.layers import BatchNormalization,MaxPool2D
from tensorflow.keras.layers import GlobalAvgPool2D
#For fully connected networks
from tensorflow.keras.models import Sequential

In [ ]:
#The code was partially taken from
# https://towardsdatascience.com/implementing-alexnet-cnn-architecture-using-tensorflow-2-0-and-keras-2113e090ad98
# Initialize the model object
model = Sequential()
# Add a convolutional layer
model.add(Conv2D(10, kernel_size=3, activation='relu', 
               input_shape=(281, 481, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Flatten the output of the convolutional layer
model.add(Flatten())
# Add an output layer for the 3 categories
model.add(Dense(1, activation='sigmoid'))

model.summary()


METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(optimizer='adam',
          loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
          metrics=METRICS)

history = model.fit(
        X_values,
        Y_values,
        epochs=1,
        validation_data=(X_val, Y_val),
        batch_size=10)
model.evaluate(X_test, Y_test)


In [ ]:
def simple_load_nc_dir_with_generator(data, generate_tensor=True):
    #def gen():
        results_list = list()
        final_dict = None
        file_path = dir_loc + data +'.nc'
        #for file in glob.glob(os.path.join(dir_, "*.nc")):
        ds = xarray.open_dataset(file_path, engine='netcdf4')

        if generate_tensor == True:
            #final_dict = {'conv2d_input': results_list.append(tf.convert_to_tensor(val)) for key, val in ds.items()}
            #results_list = [tf.convert_to_tensor(val) for key, val in ds.items()]
            results_list += list(map( lambda x : tf.convert_to_tensor(x[1]) ,
                                     ds.items() ))
        else:
            #final_dict = {'conv2d_input': results_list + [val] for key, val in ds.items()}
            results_list += list(map( lambda x : x[1],
                                     ds.items() ))

        print("CCC", ds.items())

        results_list_shape = results_list[0].shape   
        final_dict = { 'input_1': tf.reshape( results_list[0:1],
                                               shape=(results_list_shape[0],
                                                      results_list_shape[1], 
                                                      results_list_shape[2], 1)) }

        print("X # of inputs:", final_dict['input_1'].shape, "and", type(final_dict['input_1']))

        return final_dict

X_values = simple_load_nc_dir_with_generator("x_train", generate_tensor=True)
print(type(X_values))
X_val = simple_load_nc_dir_with_generator("x_val", generate_tensor=True)
X_test = simple_load_nc_dir_with_generator("x_test", generate_tensor=True)


In [ ]:
def open_csv_file(data):
   
    Y_values = list()
    with open(dir_loc + data) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:
                #print(f'\t{row[0]} and {row[1]} and {row[2]}.')
                if row[1] == "Yes":
                    Y_values.append(1)
                else:
                    Y_values.append(0)

                line_count += 1
        print(f'Processed {line_count} lines.')


    Y_values = list(map(lambda x : tf.convert_to_tensor(x), Y_values))

    Y_values = {'dense': tf.convert_to_tensor( Y_values ) }


    return Y_values
Y_values = open_csv_file("y_train.csv") 
Y_val = open_csv_file("y_val.csv") 
Y_test = open_csv_file("y_test.csv") 

In [ ]:
# creating the Conv-Batch Norm block
#This part is partially taken 
#from https://towardsdatascience.com/xception-from-scratch-using-tensorflow-even-better-than-inception-940fb231ced9

def conv_bn(x, filters, kernel_size, strides=1):
    
    x = Conv2D(filters=filters, 
               kernel_size = kernel_size, 
               strides=strides, 
               padding = 'same', 
               use_bias = False)(x)
    x = BatchNormalization()(x)
    return x
# creating separableConv-Batch Norm block

def sep_bn(x, filters, kernel_size, strides=1):
    
    x = SeparableConv2D(filters=filters, 
                        kernel_size = kernel_size, 
                        strides=strides, 
                        padding = 'same', 
                        use_bias = False)(x)
    x = BatchNormalization()(x)
    return x
# entry flow

def entry_flow(x):
    
    x = conv_bn(x, filters =32, kernel_size =3, strides=2)
    x = ReLU()(x)
    x = conv_bn(x, filters =64, kernel_size =3, strides=1)
    tensor = ReLU()(x)
    
    x = sep_bn(tensor, filters = 128, kernel_size =3)
    x = ReLU()(x)
    x = sep_bn(x, filters = 128, kernel_size =3)
    x = MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=128, kernel_size = 1,strides=2)
    x = Add()([tensor,x])
    
    x = ReLU()(x)
    x = sep_bn(x, filters =256, kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters =256, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=256, kernel_size = 1,strides=2)
    x = Add()([tensor,x])
    
    x = ReLU()(x)
    x = sep_bn(x, filters =728, kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters =728, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=728, kernel_size = 1,strides=2)
    x = Add()([tensor,x])
    return x
# middle flow

def middle_flow(tensor):
    
    for _ in range(8):
        x = ReLU()(tensor)
        x = sep_bn(x, filters = 728, kernel_size = 3)
        x = ReLU()(x)
        x = sep_bn(x, filters = 728, kernel_size = 3)
        x = ReLU()(x)
        x = sep_bn(x, filters = 728, kernel_size = 3)
        x = ReLU()(x)
        tensor = Add()([tensor,x])
        
    return tensor
# exit flow

def exit_flow(tensor):
    
    x = ReLU()(tensor)
    x = sep_bn(x, filters = 728,  kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters = 1024,  kernel_size=3)
    x = MaxPool2D(pool_size = 3, strides = 2, padding ='same')(x)
    
    tensor = conv_bn(tensor, filters =1024, kernel_size=1, strides =2)
    x = Add()([tensor,x])
    
    x = sep_bn(x, filters = 1536,  kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters = 2048,  kernel_size=3)
    x = GlobalAvgPool2D()(x)
    
    x = Dense (units = 1, activation = 'sigmoid')(x)
    
    return x
# model code

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]



input = Input(shape = (281, 481, 1))
x = entry_flow(input)
x = middle_flow(x)
output = exit_flow(x)

model = Model(inputs=input, outputs=output)
model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=METRICS)
    
history = model.fit(
            X_values,
            Y_values,
            epochs=1,
            validation_data=(X_val, Y_val),
            batch_size=10
            )
# Fit the model
model.evaluate(X_test, Y_test)
